In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from src import camvid
from src import tiramisu
from src.callbacks import PlotMetrics
from src.utils import history_to_results
from src.utils import heatmap
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
!mkdir -p models
# the location to save coarse training weights to
coarse_weights = './models/Tiramisu103-CamVid11-Aleatoric-coarse.h5'
# the location to save fine training weights to
fine_weights = './models/Tiramisu103-CamVid11-Aleatoric-fine.h5'

In [ ]:
mapping = pd.read_table('11_class.txt', sep=r'\s+', names=['og', 'new'], index_col='og')['new'].to_dict()
mapping

In [ ]:
# the size to reshape images to before transformation
target_size = (360, 480)
# the size to crop images to for coarse training
coarse_crop = (224, 224)
# the batch size during coarse training
coarse_batch = 3
# the size to crop images to for fine tune training
fine_crop = (352, 480)
# the batch size during fine training
fine_batch = 1

In [ ]:
# all crop dimensions must be divisible by this value due 
# to the requirement of equal shapes between downsampling 
# outputs and upsampling inputs imposed by the concatenation
# in skip link connections
divisible_by = int(2**5)
# iterate over all the crop dimensions
for dim in coarse_crop + fine_crop:
    # raise error if the dimension has a remainder when divided
    if dim % divisible_by:
        f = 'crop dimension ({}) must be divisible by {}'
        f = f.format(dim, divisible_by)
        raise ValueError(f)

# Coarse Tuning

In [ ]:
camvid11 = camvid.CamVid(
    mapping=mapping,
    target_size=target_size,
    crop_size=coarse_crop,
    batch_size=coarse_batch,
    horizontal_flip=True,
    ignored_labels=['Void'],
    y_repeats=1,
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y[0])
# plot the images
camvid.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
model = tiramisu.build_aleatoric_tiramisu((*coarse_crop, 3), camvid11.n,
    label_names=camvid11.discrete_to_label_map,
    class_weights=camvid11.class_mask,
    learning_rate=1e-3,
)
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_softmax_categorical_accuracy', patience=100),
    LearningRateScheduler(lambda _, lr: 0.995 * lr),
    ModelCheckpoint(coarse_weights, 
        monitor='val_softmax_categorical_accuracy', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='max'
    ),
    PlotMetrics(),
]

In [ ]:
# fit the model with the data. divide the steps per epoch by the batch size
history = model.fit_generator(generators['train'],
    epochs=200,
    steps_per_epoch=int(367 / coarse_batch),
    validation_data=generators['val'],
    validation_steps=101,
    callbacks=callbacks,
    verbose=0
)

In [ ]:
history_to_results(history)

## Test Metrics

In [ ]:
model.load_weights(coarse_weights)

In [ ]:
metrics = model.evaluate_generator(generators['test'], steps=233)
names = model.metrics_names
pd.DataFrame(metrics, names, columns=['test'])

## Qualitative Results

In [ ]:
def predict(generator) -> tuple:
    """
    Return post-processed predictions for the given generator.

    Args:
        generator: the generator to get data from

    Returns:
        a tuple of for NumPy tensors with RGB data:
        - the batch of RGB X values
        - the unmapped RGB batch of y values
        - the unmapped RGB predicted mean values from the model
        - the meatmap RGB values of the model variance

    """
    # get the batch of data
    X, y = next(generator)
    # predict mean values and variance
    u, s, _ = model.predict(X)
    s2 = s**2
    # calculate the mean variance over the labels
    s2 = plt.Normalize()(s2.mean(axis=-1))
    # return X values, unmapped y and u values, and heatmap of s2
    return X, camvid11.unmap(y[0]), camvid11.unmap(u), heatmap(s2, 'afmhot')

### Train

In [ ]:
X, y, p, s = predict(generators['train'])

In [ ]:
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
camvid.plot(X=X[1], y=y[1], y_pred=p[1], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
camvid.plot(X=X[2], y=y[2], y_pred=p[2], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

### Validation

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

### Test

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

# Clear Session

remove the current model from memory.

In [ ]:
from keras import backend as K
K.clear_session()

# Fine Tuning

In [ ]:
camvid11 = camvid.CamVid(
    mapping=mapping,
    target_size=target_size,
    crop_size=fine_crop,
    batch_size=fine_batch,
    horizontal_flip=True,
    ignored_labels=['Void'],
    y_repeats=1,
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y[0])
# plot the images
camvid.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
tune_model = tiramisu.build_aleatoric_tiramisu((*fine_crop, 3), camvid11.n,
    label_names=camvid11.discrete_to_label_map,
    class_weights=camvid11.class_mask,
    learning_rate=1e-4,
)
# load the weights from the coarsely trained model
tune_model.load_weights(coarse_weights)
tune_model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_softmax_categorical_accuracy', patience=50),
    ModelCheckpoint(fine_weights, 
        monitor='val_softmax_categorical_accuracy', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='max'
    ),
    PlotMetrics(),
]

In [ ]:
# fit the model with the data.
history = tune_model.fit_generator(generators['train'],
    epochs=200,
    steps_per_epoch=int(367 / fine_batch),
    validation_data=generators['val'],
    validation_steps=101,
    callbacks=callbacks,
    verbose=0,
)

In [ ]:
history_to_results(history)

## Test Metrics

In [ ]:
tune_model.load_weights(fine_weights)

In [ ]:
metrics = tune_model.evaluate_generator(generators['test'], steps=233)
names = tune_model.metrics_names
pd.DataFrame(metrics, names, columns=['test'])

## Qualitative Results

In [ ]:
def predict(generator) -> tuple:
    """
    Return post-processed predictions for the given generator.

    Args:
        generator: the generator to get data from

    Returns:
        a tuple of for NumPy tensors with RGB data:
        - the batch of RGB X values
        - the unmapped RGB batch of y values
        - the unmapped RGB predicted mean values from the model
        - the meatmap RGB values of the model variance

    """
    # get the batch of data
    X, y = next(generator)
    # predict mean values and variance
    u, s, _ = tune_model.predict(X)
    s2 = s**2
    # calculate the mean variance over the labels
    s2 = plt.Normalize()(s2.mean(axis=-1))
    # return X values, unmapped y and u values, and heatmap of s2
    return X, camvid11.unmap(y[0]), camvid11.unmap(u), heatmap(s2, 'afmhot')

### Train

In [ ]:
X, y, p, s = predict(generators['train'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['train'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['train'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['train'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

### Validation

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['val'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

### Test

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])

In [ ]:
X, y, p, s = predict(generators['test'])
camvid.plot(X=X[0], y=y[0], y_pred=p[0], aleatoric=s[0], order=['X', 'y', 'y_pred', 'aleatoric'])